In [ ]:
from data_loader import load_plx_session
from run_analysis import step1_load_data, step2_process_lfp
from get_path import get_path

plx_file = get_path(file_type='plx', initial_dir=r'\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF')
# Step 1-2（すでに実行済みならスキップ）
session = step1_load_data(plx_file)
lfp_result = step2_process_lfp(session)

In [ ]:


# === Step 3: ソーティング（2通りの方法） ===

# --- 方法A: GUIで新規ソーティング ---
from spike_sorting import sort_channel, bandpass_filter, SortingConfig, save_sorting_results

config = SortingConfig()
wideband = session.wideband
fs_wb = session.fs_wideband

# フィルタリング
filtered = bandpass_filter(wideband, fs_wb, config.filter_low, config.filter_high)

# Ch0だけソーティング（他のチャンネルも追加可能）
result_ch0 = sort_channel(filtered[:, 0], fs_wb, 0, config)
sorting_results = {0: result_ch0}

# GUIで手動キュレーション
from spike_sorting_gui import SpikeSortingGUI
gui = SpikeSortingGUI(sorting_results)
gui.run()
sorting_results = gui.results  # GUI終了後に取得

# 保存（次回からはこれを読み込める）
save_sorting_results(sorting_results, "sorting_ch0.npz")

# --- 方法B: v5で既に保存済みのNPZがある場合 ---
# sorting = step3_load_sorting("path/to/your/sorting.npz")

In [ ]:
# === Step 4以降（ソーティング完了後） ===
from run_analysis import step4_stimulus_analysis, step5_phase_locking, step6_save_and_plot

protocol, stim = step4_stimulus_analysis(session, sorting_results, lfp_result)
analyzer = step5_phase_locking(sorting_results, lfp_result, protocol)
step6_save_and_plot(analyzer, sorting_results, protocol, "output/")